In [50]:
import pandas as pd
from slugify import slugify
import json
import math



In [55]:
def formatINR(number):
    s, *d = str(number).partition(".")
    r = ",".join([s[x-2:x] for x in range(-3, -len(s), -2)][::-1] + [s[-3:]])
    return "".join([r] + d)

def formatValue(value, unit, type):
    if unit == "percentage" and type == "Derived":
        value = value*100
    value = round(value, 2) 
    
    if unit == "number":
        return str(formatINR(int(value)))
    elif unit == "percentage":
        return (str((formatINR(value)))+"%")
    elif unit == "rupees":
        return ("INR " + str((formatINR(value))))
    elif unit == "rupees lakhs":
        return ("INR " + str((formatINR(value))) + " Lakhs")
    else:
        return value
    
def percValue(value, unit, type):
    if unit == "percentage" and type == "Derived":
        value = value*100
    value = round(value, 2) 
    return value
    

In [44]:
# read data

raw_data = pd.read_excel('/home/abhinav/sink/D4D/data.xlsx', engine='openpyxl')
indicators = pd.read_excel('/home/abhinav/sink/D4D/Indicators.xlsx', engine='openpyxl')
dist_meta_data = {"morigaon":{"Title": "Morigaon", "About": "Morigaon district is an administrative district in the state of Assam in India. The district headquarters is located at Morigaon. The ancient place of occult Mayong is located in this district as well as Pobitora Wildlife Sanctuary.", "link": "https://morigaon.gov.in/"}, "nagaon":{"Title": "Nagaon", "About": "Nagaon is an administrative district in the Indian state of Assam. The undivided district was the most populous in the state as per 2011 census. The eastern, western, and southern fringes of the newly organized district were once ruled by different small feudal kings or their agents. It was a newly organized village system-hence called “Nagaon”, Na meaning new.", "link": "https://nagaon.assam.gov.in/"} }
dept_meta_data = {'panchayat-and-rural-development': {'Title': "Panchayat and Rural Development", 'Desc':'The Panchayat & Rural Development Department largely works to enhance livelihood opportunities, address chronic poverty, provides social security and works for economic inclusion of rural poor families. The Panchayat & Rural Development Department has been implementing a number of programmes in the rural area of the State by empowering rural poor through power of rights based law, aiming to transform the geography of poverty in the State.', 'schemes':['mgnrega']}, 
             'public-health-engineering': {'Title': "Public Health Engineering", 'Desc':'The Public Health Engineering Department has a mission to enhance quality of life of the people by ensuring sustainable safe drinking water and sanitation facilities and services along with promoting hygiene practices according to their choices and affordability.', 'schemes':['jjm']}}
scheme_meta_data = {"mgnrega": {"Name": "Mahatma Gandhi National Rural Employment Guarantee Act (MGNREGA)", "Title": "MGNREGA", "Desc": "The objective of MGNREGS is to enhance livelihood security of the rural masses. The scheme is meant to provide at least 100 days of wage employment per financial year (FY) to every rural household whose adult members are willing to do unskilled manual work. This is one of the world’s largest public sector employment programmes that provides guaranteed income through employment.", "dept_slug":"panchayat-and-rural-development", "logo": "/logo/mgnrega.webp"},
                    "jjm": {"Name": "Jal Jeevan Mission (JJM)", "Title": "JJM", "Desc": "The National Rural Drinking Water Programme / Jal Jeevan Mission is an initiative of the Government of India under the Ministry of Jal Shakti. It aims to provide safe and adequate drinking water to all households in rural India through provision of individual household tap connections by the year 2024.", "dept_slug":"public-health-engineering", "logo": "/logo/nhm.png"}}

In [47]:
#Merge indicator data into data file 
data = pd.merge(raw_data,indicators[['indicator_slug','indicator_type', 'indicator_selected', 'indicator_overview', 'indicator_explorer', 'indicator_dept', 'indicator_district', 'indicator_unit']],on='indicator_slug', how='left')


In [48]:
# filter data for district page indicators and structure the column values
data = data[data.indicator_explorer == 'yes']

def get_dept_slug(x):
    
    for key in dept_meta_data:
        if x in dept_meta_data[key]['schemes']:
            return key
        
def get_dept_name(x):
    
    for key in dept_meta_data:
        if x in dept_meta_data[key]['schemes']:
            return dept_meta_data[key]["Title"]      
    

data['district_name'] = data['district_name'].map(lambda x: x.title())
data['district_slug'] = data['district_name'].map(lambda x: slugify(x))
data['block_name'] = data['block_name'].map(lambda x: x.title())
data['block_slug'] = data['block_name'].map(lambda x: slugify(x))
data['scheme_slug'] = data['scheme_name'].map(lambda x: slugify(x))

data['dept_slug'] = data['scheme_slug'].map(lambda x: get_dept_slug(x))
data['dept_name'] = data['scheme_slug'].map(lambda x: get_dept_name(x))

data.head()

,db_code,district_name,district_code,block_name,block_code,financial_year,scheme_name,indicator_report,date_of_update,indicator_name,...,indicator_overview,indicator_explorer,indicator_dept,indicator_district,indicator_unit,district_slug,block_slug,scheme_slug,dept_slug,dept_name
140,27--1,Nagaon,27,Khagorijan,1,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-34,No of active jobcards,...,no,yes,no,yes,number,nagaon,khagorijan,mgnrega,panchayat-and-rural-development,Panchayat and Rural Development
141,27--2,Nagaon,27,Batadraba,2,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-34,No of active jobcards,...,no,yes,no,yes,number,nagaon,batadraba,mgnrega,panchayat-and-rural-development,Panchayat and Rural Development
142,27--3,Nagaon,27,Juria,3,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-34,No of active jobcards,...,no,yes,no,yes,number,nagaon,juria,mgnrega,panchayat-and-rural-development,Panchayat and Rural Development
143,27--4,Nagaon,27,Kaliabor,4,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-34,No of active jobcards,...,no,yes,no,yes,number,nagaon,kaliabor,mgnrega,panchayat-and-rural-development,Panchayat and Rural Development
144,27--5,Nagaon,27,Rupahihut,5,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-34,No of active jobcards,...,no,yes,no,yes,number,nagaon,rupahihut,mgnrega,panchayat-and-rural-development,Panchayat and Rural Development


In [49]:
data.reset_index(inplace=True)
data.head()


,index,db_code,district_name,district_code,block_name,block_code,financial_year,scheme_name,indicator_report,date_of_update,...,indicator_overview,indicator_explorer,indicator_dept,indicator_district,indicator_unit,district_slug,block_slug,scheme_slug,dept_slug,dept_name
0,140,27--1,Nagaon,27,Khagorijan,1,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-34,...,no,yes,no,yes,number,nagaon,khagorijan,mgnrega,panchayat-and-rural-development,Panchayat and Rural Development
1,141,27--2,Nagaon,27,Batadraba,2,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-34,...,no,yes,no,yes,number,nagaon,batadraba,mgnrega,panchayat-and-rural-development,Panchayat and Rural Development
2,142,27--3,Nagaon,27,Juria,3,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-34,...,no,yes,no,yes,number,nagaon,juria,mgnrega,panchayat-and-rural-development,Panchayat and Rural Development
3,143,27--4,Nagaon,27,Kaliabor,4,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-34,...,no,yes,no,yes,number,nagaon,kaliabor,mgnrega,panchayat-and-rural-development,Panchayat and Rural Development
4,144,27--5,Nagaon,27,Rupahihut,5,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-34,...,no,yes,no,yes,number,nagaon,rupahihut,mgnrega,panchayat-and-rural-development,Panchayat and Rural Development


In [56]:
# create json file for explorer page - for map and bar charts - for morigaon and nagaon

filtered_data =  data[data['district_name'].isin(["Morigaon", "Nagaon"])]

base_json = {}

for index, row in filtered_data.iterrows():
    
    if math.isinf(row.indicator_value) or math.isnan(row.indicator_value):
        continue 
    
    
    # insert district
    if  row.district_slug not in base_json:
        base_json[row.district_slug] = {"distTitle": row.district_name, "depts": {row.dept_slug: {"deptTitle": row.dept_name,
                    "schemes": {row.scheme_slug: {"schemeTitle": scheme_meta_data[row.scheme_slug]['Name'],
                    "indicators": {row.indicator_slug: {"indicatorTitle":row.indicator_name, "years": {row.financial_year: 
                        {"bardata":{[row.block_name]: [percValue(row.indicator_value, row.indicator_unit, row.indicator_report)]}, "mapdata":[{"name":int(row.BLOCK_LGD) if not math.isnan(row.BLOCK_LGD) else "Infinity", "label": row.block_name, "value": percValue(row.indicator_value, row.indicator_unit, row.indicator_report), "disp_val": formatValue(row.indicator_value, row.indicator_unit, row.indicator_report)}]}
                    }}}}} }}}
    elif row.dept_slug not in base_json[row.district_slug]["depts"]:
        base_json[row.district_slug]["depts"][row.dept_slug] = {"deptTitle": row.dept_name,
                    "schemes": {row.scheme_slug: {"schemeTitle": scheme_meta_data[row.scheme_slug]['Name'],
                    "indicators": {row.indicator_slug: {"indicatorTitle":row.indicator_name, "years": {row.financial_year: 
                        {"bardata":{[row.block_name]: [percValue(row.indicator_value, row.indicator_unit, row.indicator_report)]}, "mapdata":[{"name":int(row.BLOCK_LGD) if not math.isnan(row.BLOCK_LGD) else "Infinity", "label": row.block_name, "value": percValue(row.indicator_value, row.indicator_unit, row.indicator_report), "disp_val": formatValue(row.indicator_value, row.indicator_unit, row.indicator_report)}]}
                    }}}}} }
    elif row.scheme_slug not in base_json[row.district_slug]["depts"][row.dept_slug]["schemes"]:
        base_json[row.district_slug]["depts"][row.dept_slug]["schemes"][row.scheme_slug] = {"schemeTitle": scheme_meta_data[row.scheme_slug]['Name'],
                    "indicators": {row.indicator_slug: {"indicatorTitle":row.indicator_name, "years": {row.financial_year: 
                        {"bardata":{[row.block_name]: [percValue(row.indicator_value, row.indicator_unit, row.indicator_report)]}, "mapdata":[{"name":int(row.BLOCK_LGD) if not math.isnan(row.BLOCK_LGD) else "Infinity", "label": row.block_name, "value": percValue(row.indicator_value, row.indicator_unit, row.indicator_report), "disp_val": formatValue(row.indicator_value, row.indicator_unit, row.indicator_report)}]}
                    }}}}
    elif row.indicator_slug not in  base_json[row.district_slug]["depts"][row.dept_slug]["schemes"][row.scheme_slug]["indicators"]:
        base_json[row.district_slug]["depts"][row.dept_slug]["schemes"][row.scheme_slug]["indicators"][row.indicator_slug] = {"indicatorTitle":row.indicator_name, "years": {row.financial_year: 
                        {"bardata":{[row.block_name]: [percValue(row.indicator_value, row.indicator_unit, row.indicator_report)]}, "mapdata":[{"name":int(row.BLOCK_LGD) if not math.isnan(row.BLOCK_LGD) else "Infinity", "label": row.block_name, "value": percValue(row.indicator_value, row.indicator_unit, row.indicator_report), "disp_val": formatValue(row.indicator_value, row.indicator_unit, row.indicator_report)}]}
                    }}
    elif row.financial_year not in  base_json[row.district_slug]["depts"][row.dept_slug]["schemes"][row.scheme_slug]["indicators"][row.indicator_slug]["years"]:
        base_json[row.district_slug]["depts"][row.dept_slug]["schemes"][row.scheme_slug]["indicators"][row.indicator_slug]["years"][row.financial_year] = {"bardata":{[row.block_name]: [percValue(row.indicator_value, row.indicator_unit, row.indicator_report)]}, "mapdata":[{"name":int(row.BLOCK_LGD) if not math.isnan(row.BLOCK_LGD) else "Infinity", "label": row.block_name, "value": percValue(row.indicator_value, row.indicator_unit, row.indicator_report), "disp_val": formatValue(row.indicator_value, row.indicator_unit, row.indicator_report)}]}
    else: 
        base_json[row.district_slug]["depts"][row.dept_slug]["schemes"][row.scheme_slug]["indicators"][row.indicator_slug]["years"][row.financial_year]["bardata"]["xAxis"].append(row.block_name)
        base_json[row.district_slug]["depts"][row.dept_slug]["schemes"][row.scheme_slug]["indicators"][row.indicator_slug]["years"][row.financial_year]["bardata"]["values"].append(percValue(row.indicator_value, row.indicator_unit, row.indicator_report))
        base_json[row.district_slug]["depts"][row.dept_slug]["schemes"][row.scheme_slug]["indicators"][row.indicator_slug]["years"][row.financial_year]["mapdata"].append({"name":int(row.BLOCK_LGD) if not math.isnan(row.BLOCK_LGD) else "Infinity", "label":row.block_name, "value": percValue(row.indicator_value, row.indicator_unit, row.indicator_report), "disp_val": formatValue(row.indicator_value, row.indicator_unit, row.indicator_report)})              
    # if index > 100:
    #     break 

        
with open('scheme_explorer_data.json', 'w') as fp:
    # json.dump(base_json, fp)
    json.dump(base_json, fp, indent=2)
    



In [53]:
# create json file for explorer page - for table charts - for morigaon and nagaon

filtered_data =  data[data['district_name'].isin(["Morigaon", "Nagaon"])]

base_json = {}

for index, row in filtered_data.iterrows():
    
    if math.isinf(row.indicator_value) or math.isnan(row.indicator_value):
        continue 
    
    
    # insert district
    if  row.district_slug not in base_json:
        base_json[row.district_slug] = {"distTitle": row.district_name, "depts": {row.dept_slug: {"deptTitle": row.dept_name,
                    "schemes": {row.scheme_slug: {"schemeTitle": scheme_meta_data[row.scheme_slug]['Name'],
                    "table_data": {row.financial_year:[{"Block": row.block_name, row.indicator_name: formatValue(row.indicator_value, row.indicator_unit, row.indicator_report)}]}}}}}}
    elif row.dept_slug not in base_json[row.district_slug]["depts"]:
        base_json[row.district_slug]["depts"][row.dept_slug] = {"deptTitle": row.dept_name,
                    "schemes": {row.scheme_slug: {"schemeTitle": scheme_meta_data[row.scheme_slug]['Name'],
                    "table_data": {row.financial_year:[{"Block": row.block_name, row.indicator_name: formatValue(row.indicator_value, row.indicator_unit, row.indicator_report)}]}}}}
    elif row.scheme_slug not in base_json[row.district_slug]["depts"][row.dept_slug]["schemes"]:
        base_json[row.district_slug]["depts"][row.dept_slug]["schemes"][row.scheme_slug] =  {"schemeTitle": scheme_meta_data[row.scheme_slug]['Name'],
                    "table_data": {row.financial_year:[{"Block": row.block_name, row.indicator_name: formatValue(row.indicator_value, row.indicator_unit, row.indicator_report)}]}}
    elif row.financial_year not in  base_json[row.district_slug]["depts"][row.dept_slug]["schemes"][row.scheme_slug]["table_data"]:
        base_json[row.district_slug]["depts"][row.dept_slug]["schemes"][row.scheme_slug]["table_data"][row.financial_year] = [{"Block": row.block_name, row.indicator_name: formatValue(row.indicator_value, row.indicator_unit, row.indicator_report)}]
    else:
        i = 0 
        while i <  len(base_json[row.district_slug]["depts"][row.dept_slug]["schemes"][row.scheme_slug]["table_data"][row.financial_year]):
            if base_json[row.district_slug]["depts"][row.dept_slug]["schemes"][row.scheme_slug]["table_data"][row.financial_year][i]['Block'] == row.block_name :
                base_json[row.district_slug]["depts"][row.dept_slug]["schemes"][row.scheme_slug]["table_data"][row.financial_year][i][row.indicator_name]= formatValue(row.indicator_value, row.indicator_unit, row.indicator_report)
                break
            i+=1
        else:
            base_json[row.district_slug]["depts"][row.dept_slug]["schemes"][row.scheme_slug]["table_data"][row.financial_year].append({"Block": row.block_name, row.indicator_name: formatValue(row.indicator_value, row.indicator_unit, row.indicator_report)})  
        
    # if index > 100:
    #     break 

        
with open('scheme_explorer_table_data.json', 'w') as fp:
    # json.dump(base_json, fp)
    json.dump(base_json, fp, indent=2)

In [ ]:
# bar data structure
data: {
xAxis: [],
values: []
}


# map data structure

[
{
    name: 'blockName1',
    value: 20
  },
{
      name: 'blockName2',
    value: 30
  }
]


# table data structure
[{
    firstName: 'Tanner',
    lastName: 'Linsley',
    age: 24,
    visits: 100,
    status: 'In Relationship',
    progress: 50,
  },
  {
    firstName: 'Tandy',
    lastName: 'Miller',
    age: 40,
    visits: 40,
    status: 'Single',
    progress: 80,
  },]